In [2]:
import gym
import logging
import matplotlib.pyplot as plt

In [3]:
env = gym.make('LunarLander-v2')
env._max_episode_steps = 250
gym.logger.setLevel(logging.ERROR)

%matplotlib inline
plt.rcParams['figure.figsize'] = 20, 15
plt.rcParams["animation.html"] = "jshtml"

In [4]:
from gym.envs.box2d.lunar_lander import heuristic

In [5]:
def run_episode(env, policy):
    frames = []
    observation = env.reset()
    done = False
    while not done:
        frames.append(env.render(mode='rgb_array'))
        action = policy(env, observation)
        observation, reward, done, info = env.step(action)
    return frames

In [10]:
def heuristic_policy(env, observation):
    return heuristic(env.unwrapped, observation)
episode = run_episode(env, heuristic_policy)

In [6]:
def random_policy(env, observation):
    return env.action_space.sample()
episode = run_episode(env, random_policy)